In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
counties_info_df = pd.read_csv("../Data/counties.csv")
counties_pop_df = counties_info_df[["fips","population"]]
counties_pop_df.set_index("fips",inplace=True)
counties_pop_df.head()

population
fips            
2013        3337
2016        5634
2020      288000
2050       18386
2060         836

In [4]:
def check_inconsistencies(df):
    fips_value = df['fips'].tolist()[0]
    global error_message
    
    subset_df = df[['actuals.newCases','actuals.cases']]
    previous_cumulative_cases = 0
    previous_new_cases = 0
    for index, row in subset_df.iterrows():
        if previous_cumulative_cases + previous_new_cases > row['actuals.cases']:
            message = f"Issue at time point: {index} for fips={fips_value}"
            error_message.append(message)
        
        previous_cumulative_cases = row['actuals.cases']
        previous_new_cases = row['actuals.newCases']


In [6]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
global count_cleaned_df
global error_message
error_message= []
from tqdm import tqdm

def fill_missing_days_with_NA(df):
    df.index = pd.DatetimeIndex(df['date'].values)
    new_date_range = pd.date_range(start="2020-04-05", end="2022-01-09", freq="D")
    final_df = df.reindex(new_date_range)
    final_df['date'] = final_df.index

    return final_df


def data_preprocessing(df): 
    # columns_to_exclude = ['date', 'riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    columns_to_exclude = ['date']
    new_df = df.loc[:, ~df.columns.isin(columns_to_exclude)]
    
    #Filling missing values using backfill and forwardfill
    new_df.fillna(method="backfill", axis=None, inplace=True)
    new_df.fillna(method="ffill", axis=None, inplace=True)
    
    #Normalization using MinMaxScaler
    cols_to_norm = ['riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    scaler = MinMaxScaler()
    new_df[cols_to_norm] = scaler.fit_transform(new_df[cols_to_norm])
    
    #Copying Date
    new_df['date'] = df.date

    return new_df

def population_feature_engineering(df,fip):
    df["metrics.deathDensity"] = df['actuals.deaths']/counties_pop_df.loc[fip]["population"]
    return df


In [7]:
original_csv = pd.read_csv("../counties.finalfeatures.timeseries.csv")
original_csv.drop(['actuals.newCases','actuals.newDeaths'],axis=1,inplace=True)
fips_list = list(set(original_csv['fips'].tolist()))
county_df_list = []
# outlier_counties=[]

i=0
for fip in tqdm(fips_list):
    county_df = original_csv[original_csv['fips'] == fip]

    #STEP 1
    county_df = fill_missing_days_with_NA(county_df) 
    #dataframe is set to DateTimeIndex
    
    #STEP 2
    new_county_df = data_preprocessing(county_df)

    #STEP 3
    new_feature_engg_df = population_feature_engineering(new_county_df, fip)
    
    #STEP 4
    county_df_list.append(new_feature_engg_df) #Appending the final dataframe for each county

print(len(county_df_list))

100%|██████████| 3222/3222 [02:38<00:00, 20.31it/s]

3222


In [8]:
#Checking DataFrame before QC
baker = original_csv[original_csv['fips'] == 41001]
baker.head(15).tail(5)

date   fips state        county  actuals.cases  actuals.deaths  \
1464373  2020-03-31  41001    OR  Baker County            NaN             NaN   
1464374  2020-04-01  41001    OR  Baker County            NaN             NaN   
1464375  2020-04-02  41001    OR  Baker County            NaN             NaN   
1464376  2020-04-03  41001    OR  Baker County            NaN             NaN   
1464377  2020-05-06  41001    OR  Baker County            1.0             0.0   

         metrics.caseDensity  metrics.vaccinationsCompletedRatio  \
1464373                  0.0                                 NaN   
1464374                  0.0                                 NaN   
1464375                  0.0                                 NaN   
1464376                  0.0                                 NaN   
1464377                  0.0                                 NaN   

         metrics.vaccinationsInitiatedRatio  metrics.infectionRate  \
1464373                                 NaN                    NaN   
1464374                                 NaN                    NaN   
1464375                                 NaN                    NaN   
1464376                                 NaN                    NaN   
1464377                                 NaN                    NaN   

         riskLevels.overall  riskLevels.caseDensity  cdcTransmissionLevel  \
1464373                   0                       0                     0   
1464374                   0                       0                     0   
1464375                   0                       0                     0   
1464376                   0                       0                     0   
1464377                   0                       0                     1   

         metrics.testPositivityRatio  
1464373                        0.000  
1464374                        0.000  
1464375                        0.000  
1464376                        0.000  
1464377                        0.056

In [9]:
#Checking DataFrame after QC
county_df_list[0].head(10)

fips state        county  actuals.cases  actuals.deaths  \
2020-04-05  41001.0    OR  Baker County            1.0             0.0   
2020-04-06  41001.0    OR  Baker County            1.0             0.0   
2020-04-07  41001.0    OR  Baker County            1.0             0.0   
2020-04-08  41001.0    OR  Baker County            1.0             0.0   
2020-04-09  41001.0    OR  Baker County            1.0             0.0   
2020-04-10  41001.0    OR  Baker County            1.0             0.0   
2020-04-11  41001.0    OR  Baker County            1.0             0.0   
2020-04-12  41001.0    OR  Baker County            1.0             0.0   
2020-04-13  41001.0    OR  Baker County            1.0             0.0   
2020-04-14  41001.0    OR  Baker County            1.0             0.0   

            metrics.caseDensity  metrics.vaccinationsCompletedRatio  \
2020-04-05                  0.0                               0.007   
2020-04-06                  0.0                               0.007   
2020-04-07                  0.0                               0.007   
2020-04-08                  0.0                               0.007   
2020-04-09                  0.0                               0.007   
2020-04-10                  0.0                               0.007   
2020-04-11                  0.0                               0.007   
2020-04-12                  0.0                               0.007   
2020-04-13                  0.0                               0.007   
2020-04-14                  0.0                               0.007   

            metrics.vaccinationsInitiatedRatio  metrics.infectionRate  \
2020-04-05                                0.03                    1.0   
2020-04-06                                0.03                    1.0   
2020-04-07                                0.03                    1.0   
2020-04-08                                0.03                    1.0   
2020-04-09                                0.03                    1.0   
2020-04-10                                0.03                    1.0   
2020-04-11                                0.03                    1.0   
2020-04-12                                0.03                    1.0   
2020-04-13                                0.03                    1.0   
2020-04-14                                0.03                    1.0   

            riskLevels.overall  riskLevels.caseDensity  cdcTransmissionLevel  \
2020-04-05                 0.0                     0.0              0.333333   
2020-04-06                 0.0                     0.0              0.333333   
2020-04-07                 0.0                     0.0              0.333333   
2020-04-08                 0.0                     0.0              0.333333   
2020-04-09                 0.0                     0.0              0.333333   
2020-04-10                 0.0                     0.0              0.333333   
2020-04-11                 0.0                     0.0              0.333333   
2020-04-12                 0.0                     0.0              0.333333   
2020-04-13                 0.0                     0.0              0.333333   
2020-04-14                 0.0                     0.0              0.333333   

            metrics.testPositivityRatio       date  metrics.deathDensity  
2020-04-05                        0.056 2020-04-05                   0.0  
2020-04-06                        0.056 2020-04-06                   0.0  
2020-04-07                        0.056 2020-04-07                   0.0  
2020-04-08                        0.056 2020-04-08                   0.0  
2020-04-09                        0.056 2020-04-09                   0.0  
2020-04-10                        0.056 2020-04-10                   0.0  
2020-04-11                        0.056 2020-04-11                   0.0  
2020-04-12                        0.056 2020-04-12                   0.0  
2020-04-13                        0.056 2020-04-13        

## Concatenating list of county dataframes and Removing Outlier

In [3]:
import joblib
# joblib.dump(county_df_list, "daily_agg_counties_timeseries_list.pkl")
county_df_list = joblib.load("daily_agg_counties_timeseries_list.pkl")

In [4]:
## Sanity Check
import pandas as pd
total_rows = 0
for county in county_df_list:
    total_rows += county.shape[0]
print(total_rows)

combinedf = pd.concat(county_df_list)
print(combinedf.shape)

2078190
(2078190, 15)


In [5]:
combinedf

fips state          county  actuals.cases  actuals.deaths  \
2020-04-05  41001.0    OR    Baker County            1.0             0.0   
2020-04-06  41001.0    OR    Baker County            1.0             0.0   
2020-04-07  41001.0    OR    Baker County            1.0             0.0   
2020-04-08  41001.0    OR    Baker County            1.0             0.0   
2020-04-09  41001.0    OR    Baker County            1.0             0.0   
...             ...   ...             ...            ...             ...   
2022-01-05  24510.0    MD  Baltimore city        88200.0          1441.0   
2022-01-06  24510.0    MD  Baltimore city        89511.0          1448.0   
2022-01-07  24510.0    MD  Baltimore city        90743.0          1463.0   
2022-01-08  24510.0    MD  Baltimore city        91837.0          1469.0   
2022-01-09  24510.0    MD  Baltimore city        93703.0          1471.0   

            metrics.caseDensity  metrics.vaccinationsCompletedRatio  \
2020-04-05                  0.0                               0.007   
2020-04-06                  0.0                               0.007   
2020-04-07                  0.0                               0.007   
2020-04-08                  0.0                               0.007   
2020-04-09                  0.0                               0.007   
...                         ...                                 ...   
2022-01-05                242.1                               0.602   
2022-01-06                233.6                               0.602   
2022-01-07                228.2                               0.603   
2022-01-08                219.5                               0.603   
2022-01-09                229.4                               0.604   

            metrics.vaccinationsInitiatedRatio  metrics.infectionRate  \
2020-04-05                               0.030                    1.0   
2020-04-06                               0.030                    1.0   
2020-04-07                               0.030                    1.0   
2020-04-08                               0.030                    1.0   
2020-04-09                               0.030                    1.0   
...                                        ...                    ...   
2022-01-05                               0.687                    NaN   
2022-01-06                               0.688                    NaN   
2022-01-07                               0.687                    NaN   
2022-01-08                               0.688                    NaN   
2022-01-09                               0.689                    NaN   

            riskLevels.overall  riskLevels.caseDensity  cdcTransmissionLevel  \
2020-04-05                 0.0                     0.0              0.333333   
2020-04-06                 0.0                     0.0              0.333333   
2020-04-07                 0.0                     0.0              0.333333   
2020-04-08                 0.0                     0.0              0.333333   
2020-04-09                 0.0                     0.0              0.333333   
...                        ...                     ...                   ...   
2022-01-05                 1.0                     1.0              1.000000   
2022-01-06                 1.0                     1.0              1.000000   
2022-01-07                 1.0                     1.0              1.000000   
2022-01-08                 1.0                     1.0              1.000000   
2022-01-09                 1.0                     1.0              1.000000   

            metrics.testPositivityRatio       date  metrics.deathDensity  
2020-04-05                        0.056 2020-04-05              0.000000  
2020-04-06                        0.056 2020-04-06              0.000000  
2020-04-07                        0.056 2020-04-07              0.000000  
2020-04-08                        0.056 2020-04-08              0.000000  
2020-04-09                        0.

## Aggregation

## Features to Aggegrate by sum:
- actuals.cases
- actuals.deaths
- metrics.deathDensity
- metrics.caseDensity
- metrics.testPositivityRatio

## Features to avergae, not sum
- cdcTransmissionLevel
- riskLevels.caseDensity
- riskLevels.overall
- metrics.vaccinationsCompletedRatio
- metrics.vaccinationsInitiatedRatio

In [6]:
combinedf['date']=combinedf.index
agg_sum_df = combinedf.groupby(['date'])['actuals.cases', 'actuals.deaths', 'metrics.deathDensity', 'metrics.caseDensity'].sum()

agg_means_df = combinedf.groupby(['date'])['metrics.testPositivityRatio','cdcTransmissionLevel', 'riskLevels.caseDensity', 'riskLevels.overall',
                                          'metrics.vaccinationsCompletedRatio','metrics.vaccinationsInitiatedRatio'].mean()
agg_df = pd.merge(agg_sum_df,agg_means_df, left_index=True,right_index=True)

<ipython-input-6-066f68de8255>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_sum_df = combinedf.groupby(['date'])['actuals.cases', 'actuals.deaths', 'metrics.deathDensity', 'metrics.caseDensity'].sum()
<ipython-input-6-066f68de8255>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_means_df = combinedf.groupby(['date'])['metrics.testPositivityRatio','cdcTransmissionLevel', 'riskLevels.caseDensity', 'riskLevels.overall',


In [8]:
agg_sum_df

actuals.cases  actuals.deaths  metrics.deathDensity  \
date                                                              
2020-04-05       331769.0          9285.0              0.031711   
2020-04-06       362217.0         10662.0              0.038168   
2020-04-07       393641.0         12748.0              0.045717   
2020-04-08       425086.0         14655.0              0.050901   
2020-04-09       459307.0         16262.0              0.056929   
...                   ...             ...                   ...   
2022-01-05     56845503.0        824756.0              9.721594   
2022-01-06     57543129.0        826494.0              9.745629   
2022-01-07     58703383.0        828892.0              9.768816   
2022-01-08     59044942.0        829528.0              9.773659   
2022-01-09     59363279.0        829801.0              9.775094   

            metrics.caseDensity  
date                             
2020-04-05               9417.1  
2020-04-06              10161.4  
2020-04-07              10931.2  
2020-04-08              11404.1  
2020-04-09              11825.5  
...                         ...  
2022-01-05             351625.4  
2022-01-06             377290.1  
2022-01-07             408058.5  
2022-01-08             429196.2  
2022-01-09             447723.7  

[645 rows x 4 columns]

In [9]:
agg_means_df

metrics.testPositivityRatio  cdcTransmissionLevel  \
date                                                            
2020-04-05                     0.106803              0.448738   
2020-04-06                     0.105619              0.451557   
2020-04-07                     0.106687              0.453574   
2020-04-08                     0.106887              0.461696   
2020-04-09                     0.106429              0.463041   
...                                 ...                   ...   
2022-01-05                     0.274392              0.992086   
2022-01-06                     0.282382              0.992189   
2022-01-07                     0.288566              0.993224   
2022-01-08                     0.288564              0.993637   
2022-01-09                     0.288562              0.994051   

            riskLevels.caseDensity  riskLevels.overall  \
date                                                     
2020-04-05                0.096146            0.195474   
2020-04-06                0.100693            0.200321   
2020-04-07                0.106921            0.206631   
2020-04-08                0.109792            0.208432   
2020-04-09                0.111778            0.209332   
...                            ...                 ...   
2022-01-05                0.837409            0.840451   
2022-01-06                0.858297            0.860656   
2022-01-07                0.879847            0.881047   
2022-01-08                0.891972            0.892882   
2022-01-09                0.898955            0.899866   

            metrics.vaccinationsCompletedRatio  \
date                                             
2020-04-05                            0.018569   
2020-04-06                            0.018569   
2020-04-07                            0.018569   
2020-04-08                            0.018569   
2020-04-09                            0.018569   
...                                        ...   
2022-01-05                            0.492552   
2022-01-06                            0.493028   
2022-01-07                            0.493438   
2022-01-08                            0.493793   
2022-01-09                            0.494231   

            metrics.vaccinationsInitiatedRatio  
date                                            
2020-04-05                            0.058003  
2020-04-06                            0.058003  
2020-04-07                            0.058003  
2020-04-08                            0.058003  
2020-04-09                            0.058003  
...                                        ...  
2022-01-05                            0.555811  
2022-01-06                            0.556442  
2022-01-07                            0.557038  
2022-01-08                            0.557612  
2022-01-09                            0.557877  

[645 rows x 6 columns]

In [10]:
agg_df

actuals.cases  actuals.deaths  metrics.deathDensity  \
date                                                              
2020-04-05       331769.0          9285.0              0.031711   
2020-04-06       362217.0         10662.0              0.038168   
2020-04-07       393641.0         12748.0              0.045717   
2020-04-08       425086.0         14655.0              0.050901   
2020-04-09       459307.0         16262.0              0.056929   
...                   ...             ...                   ...   
2022-01-05     56845503.0        824756.0              9.721594   
2022-01-06     57543129.0        826494.0              9.745629   
2022-01-07     58703383.0        828892.0              9.768816   
2022-01-08     59044942.0        829528.0              9.773659   
2022-01-09     59363279.0        829801.0              9.775094   

            metrics.caseDensity  metrics.testPositivityRatio  \
date                                                           
2020-04-05               9417.1                     0.106803   
2020-04-06              10161.4                     0.105619   
2020-04-07              10931.2                     0.106687   
2020-04-08              11404.1                     0.106887   
2020-04-09              11825.5                     0.106429   
...                         ...                          ...   
2022-01-05             351625.4                     0.274392   
2022-01-06             377290.1                     0.282382   
2022-01-07             408058.5                     0.288566   
2022-01-08             429196.2                     0.288564   
2022-01-09             447723.7                     0.288562   

            cdcTransmissionLevel  riskLevels.caseDensity  riskLevels.overall  \
date                                                                           
2020-04-05              0.448738                0.096146            0.195474   
2020-04-06              0.451557                0.100693            0.200321   
2020-04-07              0.453574                0.106921            0.206631   
2020-04-08              0.461696                0.109792            0.208432   
2020-04-09              0.463041                0.111778            0.209332   
...                          ...                     ...                 ...   
2022-01-05              0.992086                0.837409            0.840451   
2022-01-06              0.992189                0.858297            0.860656   
2022-01-07              0.993224                0.879847            0.881047   
2022-01-08              0.993637                0.891972            0.892882   
2022-01-09              0.994051                0.898955            0.899866   

            metrics.vaccinationsCompletedRatio  \
date                                             
2020-04-05                            0.018569   
2020-04-06                            0.018569   
2020-04-07                            0.018569   
2020-04-08                            0.018569   
2020-04-09                            0.018569   
...                                        ...   
2022-01-05                            0.492552   
2022-01-06                            0.493028   
2022-01-07                            0.493438   
2022-01-08                            0.493793   
2022-01-09                            0.494231   

            metrics.vaccinationsInitiatedRatio  
date                                            
2020-04-05                            0.058003  
2020-04-06                            0.058003  
2020-04-07                            0.058003  
2020-04-08                            0.058003  
2020-04-09                            0.058003  
...                                        ...  
2022-01-05                            0.555811  
2022-01-06                            0.556442  
2022-01-07                            0.557038  
2022-01-08                            0.557612  
2022-01-09                 

In [11]:
agg_df.to_csv("county_daily_aggregated_timeseries.csv")